[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/jeremylongshore/claude-code-plugins-plus-skills/blob/main/tutorials/plugins/04-mcp-server-plugins.ipynb)

# MCP Server Plugins (Advanced)

**Learning Path**: Skills → Plugins → **MCP**  
**Level**: Advanced  
**Time**: 60 minutes  
**Prerequisites**: [01-what-is-plugin](01-what-is-plugin.ipynb), [02-plugin-structure](02-plugin-structure.ipynb), [03-build-your-first-plugin](03-build-your-first-plugin.ipynb)

---

## What You'll Learn

1. ✅ **What is MCP** - Model Context Protocol overview
2. ✅ **When to use MCP** - vs instruction plugins
3. ✅ **TypeScript setup** - Project structure and build
4. ✅ **Tool implementation** - Create custom tools with Zod validation
5. ✅ **Server lifecycle** - Initialization and request handling
6. ✅ **Testing** - Validate your MCP server
7. ✅ **Real example** - Build a task-tracker MCP plugin

---

## What is MCP?

**Model Context Protocol (MCP)**: Standard protocol for connecting LLMs to external tools and data sources.

### Instruction Plugin vs MCP Plugin

| Aspect | Instruction Plugin | MCP Plugin |
|--------|-------------------|------------|
| **Language** | Markdown (YAML frontmatter) | TypeScript/Node.js |
| **Execution** | Claude's built-in tools only | Custom code execution |
| **External APIs** | ❌ No | ✅ Yes |
| **Database** | ❌ No | ✅ Yes |
| **File System** | Limited (Read/Write tools) | ✅ Full access |
| **Complexity** | Low (easy to create) | High (requires TypeScript) |
| **Use Case** | Guidance & workflows | Data access & integrations |

### When to Use MCP

✅ **Use MCP when you need**:
- External API calls (REST, GraphQL)
- Database queries (SQL, NoSQL)
- File system operations beyond Read/Write
- Real-time data streaming
- Custom computation (crypto, image processing)

❌ **Don't use MCP for**:
- Simple workflows (use instruction plugins)
- Claude's built-in tools are sufficient
- No external integrations needed

In [ ]:
# MCP vs Instruction decision tree
def should_use_mcp(requirements):
    """
    Decision helper for MCP vs Instruction plugin.
    """
    needs_mcp = [
        requirements.get('external_api', False),
        requirements.get('database_access', False),
        requirements.get('file_system_ops', False),
        requirements.get('realtime_data', False),
        requirements.get('custom_computation', False)
    ]
    
    if any(needs_mcp):
        return {
            'recommendation': 'MCP Server Plugin',
            'reason': 'Requires code execution for ' + ', '.join([
                k.replace('_', ' ') for k, v in requirements.items() if v
            ]),
            'complexity': 'High',
            'languages': ['TypeScript', 'Node.js']
        }
    else:
        return {
            'recommendation': 'Instruction Plugin',
            'reason': 'No external integrations needed',
            'complexity': 'Low',
            'languages': ['Markdown', 'YAML']
        }

# Test cases
examples = [
    {'name': 'Code reviewer', 'external_api': False, 'database_access': False},
    {'name': 'Stripe payments', 'external_api': True, 'database_access': False},
    {'name': 'SQLite browser', 'external_api': False, 'database_access': True},
]

print("MCP DECISION HELPER")
print("=" * 60)
for ex in examples:
    name = ex.pop('name')
    result = should_use_mcp(ex)
    print(f"\n{name}:")
    print(f"  → {result['recommendation']}")
    print(f"  Why: {result['reason']}")
    print(f"  Complexity: {result['complexity']}")

## MCP Architecture

### Components

```
Claude Code (Client)
       |
       | JSON-RPC over stdio
       |
   MCP Server
       |
       |-- Tools (functions Claude can call)
       |-- Resources (data Claude can access)
       └-- Prompts (templates)
```

### Communication Flow

1. **Claude calls tool**: `{"method": "tools/call", "params": {"name": "create_task", "arguments": {...}}}`
2. **MCP server validates**: Use Zod schemas to check arguments
3. **MCP server executes**: Run custom TypeScript code
4. **MCP server responds**: `{"result": {...}}`
5. **Claude receives result**: Continue conversation with returned data

### Transport

**StdioServerTransport**: JSON-RPC messages over stdin/stdout
- No network required
- Secure (local process)
- Fast (IPC)

---

## Project Setup

### Directory Structure

```
task-tracker-mcp/
├── .claude-plugin/
│   └── plugin.json
├── servers/
│   └── task-tracker.ts      ← MCP server code
├── dist/
│   └── servers/
│       └── task-tracker.js  ← Compiled output
├── package.json
├── tsconfig.json
├── README.md
└── LICENSE
```

### Dependencies

```json
{
  "dependencies": {
    "@modelcontextprotocol/sdk": "^1.21.1",
    "zod": "^4.1.12",
    "zod-to-json-schema": "^3.25.0"
  },
  "devDependencies": {
    "@types/node": "^20.19.25",
    "typescript": "^5.6.0"
  }
}
```

In [ ]:
import json
from pathlib import Path

# Generate package.json for MCP plugin
package_json = {
    "name": "@your-org/task-tracker-mcp",
    "version": "1.0.0",
    "description": "Task tracker MCP server for managing todos",
    "type": "module",
    "main": "./dist/servers/task-tracker.js",
    "scripts": {
        "build": "tsc",
        "dev": "tsc --watch",
        "test": "node dist/servers/task-tracker.js"
    },
    "keywords": ["mcp", "tasks", "todo", "productivity"],
    "author": {"name": "Your Name", "email": "you@example.com"},
    "license": "MIT",
    "dependencies": {
        "@modelcontextprotocol/sdk": "^1.21.1",
        "zod": "^4.1.12",
        "zod-to-json-schema": "^3.25.0"
    },
    "devDependencies": {
        "@types/node": "^20.19.25",
        "typescript": "^5.6.0"
    }
}

# TypeScript configuration
tsconfig = {
    "compilerOptions": {
        "target": "ES2022",
        "module": "Node16",
        "moduleResolution": "Node16",
        "outDir": "./dist",
        "rootDir": "./",
        "strict": True,
        "esModuleInterop": True,
        "skipLibCheck": True,
        "forceConsistentCasingInFileNames": True,
        "declaration": True
    },
    "include": ["servers/**/*"],
    "exclude": ["node_modules", "dist"]
}

print("PACKAGE.JSON")
print("=" * 60)
print(json.dumps(package_json, indent=2))

print("\n\nTSCONFIG.JSON")
print("=" * 60)
print(json.dumps(tsconfig, indent=2))

## Building the MCP Server

### Complete TypeScript Implementation

```typescript
#!/usr/bin/env node
/**
 * Task Tracker MCP Server
 * Manage tasks with create, list, update, delete operations
 */

import { Server } from '@modelcontextprotocol/sdk/server/index.js';
import { StdioServerTransport } from '@modelcontextprotocol/sdk/server/stdio.js';
import {
  CallToolRequestSchema,
  ListToolsRequestSchema,
  Tool
} from '@modelcontextprotocol/sdk/types.js';
import { z } from 'zod';
import { zodToJsonSchema } from 'zod-to-json-schema';

// Data models
interface Task {
  id: string;
  title: string;
  description?: string;
  status: 'todo' | 'in_progress' | 'done';
  priority: 'low' | 'medium' | 'high';
  createdAt: string;
  updatedAt: string;
}

// In-memory storage
const tasks = new Map<string, Task>();

// Zod schemas for validation
const CreateTaskSchema = z.object({
  title: z.string().min(1).max(200),
  description: z.string().optional(),
  priority: z.enum(['low', 'medium', 'high']).default('medium')
});

const ListTasksSchema = z.object({
  status: z.enum(['todo', 'in_progress', 'done']).optional(),
  priority: z.enum(['low', 'medium', 'high']).optional()
});

const UpdateTaskSchema = z.object({
  taskId: z.string(),
  title: z.string().optional(),
  description: z.string().optional(),
  status: z.enum(['todo', 'in_progress', 'done']).optional(),
  priority: z.enum(['low', 'medium', 'high']).optional()
});

const DeleteTaskSchema = z.object({
  taskId: z.string()
});

// Tool implementations
async function createTask(args: z.infer<typeof CreateTaskSchema>) {
  const taskId = `task_${Date.now()}`;
  const now = new Date().toISOString();
  
  const task: Task = {
    id: taskId,
    title: args.title,
    description: args.description,
    status: 'todo',
    priority: args.priority,
    createdAt: now,
    updatedAt: now
  };
  
  tasks.set(taskId, task);
  
  return {
    taskId,
    message: `Created task: ${task.title}`,
    task
  };
}

async function listTasks(args: z.infer<typeof ListTasksSchema>) {
  let filtered = Array.from(tasks.values());
  
  if (args.status) {
    filtered = filtered.filter(t => t.status === args.status);
  }
  
  if (args.priority) {
    filtered = filtered.filter(t => t.priority === args.priority);
  }
  
  return {
    total: filtered.length,
    tasks: filtered
  };
}

async function updateTask(args: z.infer<typeof UpdateTaskSchema>) {
  const task = tasks.get(args.taskId);
  if (!task) {
    throw new Error(`Task not found: ${args.taskId}`);
  }
  
  const updated = {
    ...task,
    ...args,
    id: task.id, // Don't allow ID change
    updatedAt: new Date().toISOString()
  };
  
  tasks.set(args.taskId, updated);
  
  return {
    message: 'Task updated',
    task: updated
  };
}

async function deleteTask(args: z.infer<typeof DeleteTaskSchema>) {
  const task = tasks.get(args.taskId);
  if (!task) {
    throw new Error(`Task not found: ${args.taskId}`);
  }
  
  tasks.delete(args.taskId);
  
  return {
    message: `Deleted task: ${task.title}`,
    deletedTask: task
  };
}

// Server setup
const server = new Server(
  {
    name: 'task-tracker',
    version: '1.0.0'
  },
  {
    capabilities: {
      tools: {}
    }
  }
);

// Register tools
server.setRequestHandler(ListToolsRequestSchema, async () => {
  return {
    tools: [
      {
        name: 'create_task',
        description: 'Create a new task',
        inputSchema: zodToJsonSchema(CreateTaskSchema)
      },
      {
        name: 'list_tasks',
        description: 'List tasks with optional filters',
        inputSchema: zodToJsonSchema(ListTasksSchema)
      },
      {
        name: 'update_task',
        description: 'Update an existing task',
        inputSchema: zodToJsonSchema(UpdateTaskSchema)
      },
      {
        name: 'delete_task',
        description: 'Delete a task',
        inputSchema: zodToJsonSchema(DeleteTaskSchema)
      }
    ] as Tool[]
  };
});

// Handle tool calls
server.setRequestHandler(CallToolRequestSchema, async (request) => {
  const { name, arguments: args } = request.params;
  
  try {
    let result;
    
    switch (name) {
      case 'create_task':
        result = await createTask(CreateTaskSchema.parse(args));
        break;
      case 'list_tasks':
        result = await listTasks(ListTasksSchema.parse(args));
        break;
      case 'update_task':
        result = await updateTask(UpdateTaskSchema.parse(args));
        break;
      case 'delete_task':
        result = await deleteTask(DeleteTaskSchema.parse(args));
        break;
      default:
        throw new Error(`Unknown tool: ${name}`);
    }
    
    return {
      content: [
        {
          type: 'text',
          text: JSON.stringify(result, null, 2)
        }
      ]
    };
  } catch (error) {
    return {
      content: [
        {
          type: 'text',
          text: `Error: ${error instanceof Error ? error.message : String(error)}`
        }
      ],
      isError: true
    };
  }
});

// Start server
async function main() {
  const transport = new StdioServerTransport();
  await server.connect(transport);
  console.error('Task Tracker MCP server running on stdio');
}

main().catch(console.error);
```

## Key Concepts

### 1. Zod Validation

**Why Zod?**: Runtime type safety for untrusted input

```typescript
const CreateTaskSchema = z.object({
  title: z.string().min(1).max(200),  // Required, 1-200 chars
  priority: z.enum(['low', 'medium', 'high']).default('medium')
});

// Parse and validate
const validated = CreateTaskSchema.parse(userInput);
// TypeScript knows validated.title is a string
```

---

### 2. Tool Registration

```typescript
server.setRequestHandler(ListToolsRequestSchema, async () => {
  return {
    tools: [
      {
        name: 'create_task',  // Tool identifier
        description: 'Create a new task',  // Claude sees this
        inputSchema: zodToJsonSchema(CreateTaskSchema)  // JSON Schema
      }
    ]
  };
});
```

**Flow**:
1. Claude requests available tools
2. Server returns tool list with schemas
3. Claude chooses tools based on descriptions
4. Server validates arguments with Zod

---

### 3. Error Handling

```typescript
try {
  result = await createTask(CreateTaskSchema.parse(args));
} catch (error) {
  return {
    content: [{ type: 'text', text: `Error: ${error.message}` }],
    isError: true  // Claude knows this failed
  };
}
```

**Best practices**:
- Always catch errors
- Return helpful error messages
- Set `isError: true` flag
- Log errors to stderr (not stdout)

In [ ]:
# Simulate MCP tool call flow
def simulate_mcp_call(tool_name, arguments):
    """
    Simulate MCP server request/response.
    """
    print(f"CLIENT → SERVER: Call tool '{tool_name}'")
    print(f"Arguments: {arguments}")
    print("\nSERVER:")
    print("  1. Validate arguments with Zod")
    print("  2. Execute tool function")
    print("  3. Return result\n")
    
    # Simulate validation
    if tool_name == "create_task":
        if "title" not in arguments:
            return {"error": "Missing required field: title", "isError": True}
        
        result = {
            "taskId": "task_123",
            "message": f"Created task: {arguments['title']}",
            "task": {
                "id": "task_123",
                "title": arguments['title'],
                "status": "todo",
                "priority": arguments.get('priority', 'medium')
            }
        }
        print(f"SERVER → CLIENT: Success")
        return result
    
    return {"error": f"Unknown tool: {tool_name}", "isError": True}

# Test the flow
print("MCP TOOL CALL SIMULATION")
print("=" * 60)
result = simulate_mcp_call("create_task", {
    "title": "Build MCP plugin",
    "priority": "high"
})

print("\nRESULT:")
import json
print(json.dumps(result, indent=2))

## Building and Testing

### Build Process

```bash
# Install dependencies
npm install

# Compile TypeScript → JavaScript
npm run build

# Output: dist/servers/task-tracker.js
```

### Make Executable

```bash
chmod +x dist/servers/task-tracker.js
```

The shebang `#!/usr/bin/env node` at top of file makes it executable.

### Testing the Server

```bash
# Test server starts
node dist/servers/task-tracker.js

# Should output to stderr:
# Task Tracker MCP server running on stdio
```

### Integration with Claude

Add to `.claude-plugin/plugin.json`:

```json
{
  "name": "task-tracker-mcp",
  "version": "1.0.0",
  "description": "Task management via MCP",
  "mcpServers": {
    "task-tracker": {
      "command": "node",
      "args": ["${CLAUDE_PLUGIN_ROOT}/dist/servers/task-tracker.js"]
    }
  }
}
```

**Portable path**: `${CLAUDE_PLUGIN_ROOT}` expands to plugin directory

In [ ]:
# Generate complete plugin.json for MCP plugin
plugin_json_mcp = {
    "name": "task-tracker-mcp",
    "version": "1.0.0",
    "description": "Task management MCP server with CRUD operations",
    "author": {
        "name": "Your Name",
        "email": "you@example.com"
    },
    "license": "MIT",
    "keywords": ["mcp", "tasks", "todo", "productivity"],
    "category": "productivity",
    "mcpServers": {
        "task-tracker": {
            "command": "node",
            "args": ["${CLAUDE_PLUGIN_ROOT}/dist/servers/task-tracker.js"],
            "env": {}
        }
    }
}

print("COMPLETE PLUGIN.JSON FOR MCP")
print("=" * 60)
print(json.dumps(plugin_json_mcp, indent=2))

print("\n✅ Key points:")
print("  - Uses ${CLAUDE_PLUGIN_ROOT} for portability")
print("  - Points to compiled dist/ output")
print("  - Empty env object (add secrets here if needed)")

## Key Takeaways

### What You Learned

1. ✅ **MCP basics** - Model Context Protocol for external integrations
2. ✅ **TypeScript setup** - package.json, tsconfig, dependencies
3. ✅ **Tool creation** - Zod schemas, validation, error handling
4. ✅ **Server lifecycle** - Initialization, request handling, transport
5. ✅ **Real example** - Complete task-tracker implementation
6. ✅ **Integration** - plugin.json configuration, portable paths

### MCP Best Practices

- **Always validate** - Use Zod for runtime type safety
- **Handle errors** - Return helpful messages with `isError` flag
- **Portable paths** - Use `${CLAUDE_PLUGIN_ROOT}` in configs
- **Shebang required** - `#!/usr/bin/env node` for executable
- **Stderr for logs** - stdout is for JSON-RPC only
- **Build before use** - Compile TypeScript → JavaScript

### When to Use MCP

✅ **Use MCP**:
- External APIs (REST, GraphQL, gRPC)
- Database queries (SQL, NoSQL, ORMs)
- File system beyond Read/Write
- Real-time data streams
- Custom computation (crypto, ML, image processing)

❌ **Use Instruction Plugin**:
- Workflows and guidance
- Claude's built-in tools sufficient
- No external dependencies
- Simple automations

---

## Next Steps

1. **Add more tools** - Expand task-tracker with search, tags, due dates
2. **Persistent storage** - Replace Map with SQLite or file storage
3. **Error recovery** - Handle crashes, validate state
4. **Testing** - Write unit tests with Vitest
5. **Publish** - Share on npm or marketplace

### Example Extensions

- **Stripe MCP**: Payment processing
- **Postgres MCP**: Database queries
- **API Client MCP**: REST API wrapper
- **File Search MCP**: Advanced file operations

---

*Enterprise Standards Compliant • 6767-c • Version 1.0.0 • MIT License*